In [ ]:
# Now this is the code made for Kaggle environment, won't run in local.

In [1]:
import os

import pandas as pd
import polars as pl

import kaggle_evaluation.cmi_inference_server

In [2]:
# Importing needed libraries
import numpy as np
import pandas as pd
import sklearn as sk

The evaluation API requires that you set up a server which will respond to inference requests. We have already defined the server; you just need write the predict function. When we evaluate your submission on the hidden test set the client defined in the gateway will run in a different container with direct access to the hidden test set and hand off the one sequence at a time.

Your code will always have access to the published copies of the files.

In [3]:
'''
def predict(sequence: pl.DataFrame, demographics: pl.DataFrame) -> str:
    # Replace this function with your inference code.
    # You can return either a Pandas or Polars dataframe, though Polars is recommended.
    # Each prediction (except the very first) must be returned within 30 minutes of the batch features being provided.
    return 'Text on phone'
'''

"\ndef predict(sequence: pl.DataFrame, demographics: pl.DataFrame) -> str:\n    # Replace this function with your inference code.\n    # You can return either a Pandas or Polars dataframe, though Polars is recommended.\n    # Each prediction (except the very first) must be returned within 30 minutes of the batch features being provided.\n    return 'Text on phone'\n"

In [4]:
# Here first row from test data is taken and prediction returned, 
# the second time the funtion is called the second row of test data is taken and prediction returned and so on.

# Global counter to keep track of which test row to use
current_test_index = 0

def predict(sequence: pl.DataFrame, demographics: pl.DataFrame) -> str:
    global current_test_index
    
    # Loading the CSV file
    file_path = '/kaggle/input/cmi-detect-behavior-with-sensor-data/train.csv'
    
    # Reading the CSV file
    df_original_data = pd.read_csv(file_path)

    # Display the first few rows of the dataframe, we will be doing this as checking step every step along the way
    #df_original_data.head()

    # Checking number of null values in each column
    #df_original_data.isnull().sum()

    # Defining feature and target
    A = df_original_data.iloc[:, 9:].values
    b = df_original_data.iloc[:, 8].values

    #print(A)
    #print(b)

    # Now the feature column has some null values - imputing them with a constant value Zero    
    # Replace NaNs with 0
    A = df_original_data.iloc[:, 9:].values
    A = np.nan_to_num(A, nan=0)
    #print(A[:5])  # Display first 5 rows

    # Scaling
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    A_train_s = sc.fit_transform(A)

    # Instead of test train split, using whole of the data to train our model.

    # Fitting KNN to the training set

    from sklearn.neighbors import KNeighborsClassifier
    classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
    classifier.fit(A_train_s, b)

    # Importing the testing data 

    # Convert Polars DataFrame to Pandas 
    df_original_test_data = sequence.to_pandas()

    # Display the first few rows of the dataframe, we will be doing this as checking step every step along the way
    #df_original_test_data.head()

    print(current_test_index)
    # Only taking needed columns from test data, and the needed row
    original_test_data = df_original_test_data.iloc[current_test_index, 4:].values
    sequence_id = df_original_test_data['sequence_id'].iloc[current_test_index]
    print(sequence_id) # To show current sequence_id data being tested out.
    current_test_index += 1 # To update the row value, when the funtion is called next time
    
    # Reshape is done coz above code gives a 1D shape while the input for next is a 2D shape, just a small tweak.
    original_test_data = original_test_data.reshape(1, -1)
    #print(original_test_data)

    # Predicting the test set - results
    y_pred = classifier.predict(original_test_data)
    #print(y_pred)

    y_pred_string = str(y_pred[0]) # Converting the return value to a string.
    #print(y_pred_string)
    print(sequence_id, y_pred_string)
    
    print('Checkpoint 1 ~ Akash')
    return y_pred_string

    # ~ End of code.

In [5]:
'''
inference_server = kaggle_evaluation.cmi_inference_server.CMIInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(
        data_paths=(
            '/kaggle/input/cmi-detect-behavior-with-sensor-data/test.csv',
            '/kaggle/input/cmi-detect-behavior-with-sensor-data/test_demographics.csv',
        )
    )
'''

"\ninference_server = kaggle_evaluation.cmi_inference_server.CMIInferenceServer(predict)\n\nif os.getenv('KAGGLE_IS_COMPETITION_RERUN'):\n    inference_server.serve()\nelse:\n    inference_server.run_local_gateway(\n        data_paths=(\n            '/kaggle/input/cmi-detect-behavior-with-sensor-data/test.csv',\n            '/kaggle/input/cmi-detect-behavior-with-sensor-data/test_demographics.csv',\n        )\n    )\n"

In [6]:
# Loading the CSV file
file_path_test = '/kaggle/input/cmi-detect-behavior-with-sensor-data/test.csv'
    
# Reading the CSV file
df_original_test_data = pd.read_csv(file_path_test)

# Get number of rows in test data
num_rows = df_original_test_data.shape[0]
print(num_rows)

print('sequence_id, prediction')

# Call predict() for each row
for i in range(num_rows):    
    inference_server = kaggle_evaluation.cmi_inference_server.CMIInferenceServer(predict)

    if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
        print('Checkpoint 2 ~ Akash')
        inference_server.serve()
    else:
        print('Checkpoint 2.5 ~ Akash')
        inference_server.run_local_gateway(
            data_paths=(
                '/kaggle/input/cmi-detect-behavior-with-sensor-data/test.csv',
                '/kaggle/input/cmi-detect-behavior-with-sensor-data/test_demographics.csv',
            )
        )
    print('Checkpoint 3 ~ Akash')

107
sequence_id, prediction
Checkpoint 2.5 ~ Akash
0
SEQ_000011
SEQ_000011 Pull air toward your face
Checkpoint 1 ~ Akash
1
SEQ_000001
SEQ_000001 Wave hello
Checkpoint 1 ~ Akash
Checkpoint 3 ~ Akash
Checkpoint 2.5 ~ Akash
2
SEQ_000001
SEQ_000001 Wave hello
Checkpoint 1 ~ Akash
3
SEQ_000011
SEQ_000011 Text on phone
Checkpoint 1 ~ Akash
Checkpoint 3 ~ Akash
Checkpoint 2.5 ~ Akash
4
SEQ_000001
SEQ_000001 Wave hello
Checkpoint 1 ~ Akash
5
SEQ_000011
SEQ_000011 Text on phone
Checkpoint 1 ~ Akash
Checkpoint 3 ~ Akash
Checkpoint 2.5 ~ Akash
6
SEQ_000001
SEQ_000001 Wave hello
Checkpoint 1 ~ Akash
7
SEQ_000011
SEQ_000011 Text on phone
Checkpoint 1 ~ Akash
Checkpoint 3 ~ Akash
Checkpoint 2.5 ~ Akash
8
SEQ_000001
SEQ_000001 Wave hello
Checkpoint 1 ~ Akash
9
SEQ_000011
SEQ_000011 Wave hello
Checkpoint 1 ~ Akash
Checkpoint 3 ~ Akash
Checkpoint 2.5 ~ Akash
10
SEQ_000011
SEQ_000011 Pull air toward your face
Checkpoint 1 ~ Akash
11
SEQ_000001
SEQ_000001 Wave hello
Checkpoint 1 ~ Akash
Checkpoint 3 ~ A

/kaggle/input/cmi-detect-behavior-with-sensor-data/kaggle_evaluation/core/templates.py:136: RuntimeWarning: 924 seconds elapsed before server startup.
                This exceeds the startup time limit of 900 seconds that the gateway will enforce
                during the rerun on the hidden test set. Start the server before performing any time consuming steps.
  warnings.warn(


28
SEQ_000011
SEQ_000011 Forehead - pull hairline
Checkpoint 1 ~ Akash
29
SEQ_000001
SEQ_000001 Cheek - pinch skin
Checkpoint 1 ~ Akash
Checkpoint 3 ~ Akash
Checkpoint 2.5 ~ Akash
30
SEQ_000001
SEQ_000001 Cheek - pinch skin
Checkpoint 1 ~ Akash
31
SEQ_000011
SEQ_000011 Forehead - pull hairline
Checkpoint 1 ~ Akash
Checkpoint 3 ~ Akash
Checkpoint 2.5 ~ Akash
32
SEQ_000011
SEQ_000011 Forehead - pull hairline
Checkpoint 1 ~ Akash
33
SEQ_000001
SEQ_000001 Cheek - pinch skin
Checkpoint 1 ~ Akash
Checkpoint 3 ~ Akash
Checkpoint 2.5 ~ Akash
34
SEQ_000011
SEQ_000011 Forehead - pull hairline
Checkpoint 1 ~ Akash
35
SEQ_000001
SEQ_000001 Cheek - pinch skin
Checkpoint 1 ~ Akash
Checkpoint 3 ~ Akash
Checkpoint 2.5 ~ Akash
36
SEQ_000011
SEQ_000011 Pull air toward your face
Checkpoint 1 ~ Akash
37
SEQ_000001
SEQ_000001 Cheek - pinch skin
Checkpoint 1 ~ Akash
Checkpoint 3 ~ Akash
Checkpoint 2.5 ~ Akash
38
SEQ_000011
SEQ_000011 Forehead - pull hairline
Checkpoint 1 ~ Akash
39
SEQ_000001
SEQ_000001 Che

GatewayRuntimeError: (<GatewayRuntimeErrorType.SERVER_RAISED_EXCEPTION: 3>, 'single positional indexer is out-of-bounds')